In [1]:
import os
import json
import os.path
if os.path.basename(os.getcwd()) == "notebooks":
    os.chdir('..')
from dotenv import load_dotenv

load_dotenv(".env")  # take environment variables from .env.

from steam_trade_bot.containers import Container
from steam_trade_bot.settings import BotSettings
container = Container()
container.config.from_pydantic(BotSettings())
container.wire(modules=[__name__])

In [2]:
uow_ = container.repositories.unit_of_work

In [57]:
async with uow_() as uow:
    history = []
    for game in await uow.game.get_all():
        history.extend(await uow.sell_history.get_all(app_id=game.app_id, currency=1, count=500000))

In [58]:
len(history)

330277

In [70]:
import csv
import pandas as pd
from steam_trade_bot.domain.services.sell_history_analyzer import steam_date_str_to_datetime

# Write CSV file
with open("bigdata.csv", "wt", newline='', encoding="utf8") as fp:
    writer = csv.writer(fp, delimiter=",")
    writer.writerow(['app_id', 'market_hash_name', 'timestamp', 'price', 'amount'])  # write header
    for i, element in enumerate(history):
        j = json.loads(element.history)
        writer.writerows([(element.app_id, element.market_hash_name, steam_date_str_to_datetime(pair[0]), round(pair[1], 2), int(pair[2])) for pair in j])
        if i % 1000 == 0:
            print(f"Processed {i} items")

Processed 0 items
Processed 1000 items
Processed 2000 items
Processed 3000 items
Processed 4000 items
Processed 5000 items
Processed 6000 items
Processed 7000 items
Processed 8000 items
Processed 9000 items
Processed 10000 items
Processed 11000 items
Processed 12000 items
Processed 13000 items
Processed 14000 items
Processed 15000 items
Processed 16000 items
Processed 17000 items
Processed 18000 items
Processed 19000 items
Processed 20000 items
Processed 21000 items
Processed 22000 items
Processed 23000 items
Processed 24000 items
Processed 25000 items
Processed 26000 items
Processed 27000 items
Processed 28000 items
Processed 29000 items
Processed 30000 items
Processed 31000 items
Processed 32000 items
Processed 33000 items
Processed 34000 items
Processed 35000 items
Processed 36000 items
Processed 37000 items
Processed 38000 items
Processed 39000 items
Processed 40000 items
Processed 41000 items
Processed 42000 items
Processed 43000 items
Processed 44000 items
Processed 45000 items
P

In [4]:
df = pd.DataFrame([], columns=['app_id', 'market_hash_name', 'timestamp', 'price', 'amount'])

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   app_id            0 non-null      object
 1   market_hash_name  0 non-null      object
 2   timestamp         0 non-null      object
 3   price             0 non-null      object
 4   amount            0 non-null      object
dtypes: object(5)
memory usage: 0.0+ bytes


In [5]:
df = pd.read_csv("bigdata.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 215911515 entries, 0 to 215911514
Data columns (total 5 columns):
 #   Column            Dtype  
---  ------            -----  
 0   app_id            int64  
 1   market_hash_name  object 
 2   timestamp         object 
 3   price             float64
 4   amount            int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 8.0+ GB


In [8]:
df["app_id"].value_counts()

753        123424132
570         33247132
730         26171717
440         12111915
218620       4315816
             ...    
1818410            2
1706770            2
1680550            2
1412190            2
1187920            2
Name: app_id, Length: 248, dtype: int64

In [21]:
df["partition"] = df.apply(lambda x: x["timestamp"].rsplit('-', 1)[0], axis=1)

In [22]:
df.to_parquet(path='./parquet/', engine='fastparquet', compression='snappy', index=False, partition_cols=["partition"])

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 215911515 entries, 0 to 215911514
Data columns (total 6 columns):
 #   Column            Dtype  
---  ------            -----  
 0   app_id            int64  
 1   market_hash_name  object 
 2   timestamp         object 
 3   price             float64
 4   amount            int64  
 5   partition         object 
dtypes: float64(1), int64(2), object(3)
memory usage: 9.7+ GB


In [24]:
del df
